In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import anndata
import string
import gc
from anndata import read_h5ad
from anndata import read_csv
from pandas import DataFrame
import h5py
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import networkx as nx
import plotly.graph_objects as go
import scanpy as sc
from cnmf import cNMF
import pickle

/home/anton.thieme/miniconda3/envs/sc_env_cnmf/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)


## PREPARE DATA

In [2]:
adata_tf = read_h5ad(os.path.join('..', 'data', 'adata_tf.h5ad'))
adata_tf

AnnData object with n_obs × n_vars = 1093048 × 1637
    obs: 'donor', 'tissue', 'anatomical_position', 'method', 'cdna_plate', 'library_plate', 'notes', 'cdna_well', 'old_index', 'assay', 'sample_id', 'replicate', '10X_run', '10X_barcode', 'ambient_removal', 'donor_method', 'donor_assay', 'donor_tissue', 'donor_tissue_assay', 'cell_ontology_class', 'cell_ontology_id', 'compartment', 'broad_cell_class', 'free_annotation', 'manually_annotated', 'published_2022', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ercc', 'pct_counts_ercc', '_scvi_batch', '_scvi_labels', 'scvi_leiden_donorassay_full', 'age', 'sex', 'ethnicity', 'sample_number'
    var: 'ensembl_id', 'gene_symbol', 'genome', 'mt', 'ercc', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: '_scvi_manager_uuid', '_scvi_uuid', '_training_mode', 'age_colors', 'assay_colors', 'compartment_colors', 'donor_colors', 'leiden', 'log1p', 'method_colors

In [ ]:
for donor in adata_tf.obs['donor'].unique():
    print(donor)
    adata_tf_donor = adata_tf[adata_tf.obs['donor'] == donor]
    for tissue in adata_tf_donor.obs['tissue'].unique():
        print(tissue)
        adata_tf_donor_tissue = adata_tf_donor[adata_tf_donor.obs['tissue'] == tissue]
        for celltype in adata_tf_donor_tissue.obs['broad_cell_class'].unique():
            print(celltype)
            adata_tf_donor_tissue_celltype = adata_tf_donor_tissue[adata_tf_donor_tissue.obs['broad_cell_class'] == celltype]
            adata_tf_donor_tissue_celltype.write_h5ad(os.path.join('..', 'data', 'donor_tissue_celltype_split', 'adata_tf_' + donor + '_' + tissue + '_' + celltype + '.h5ad'))
            del adata_tf_donor_tissue_celltype
            gc.collect()

In [ ]:
for celltype in adata_tf.obs['broad_cell_class'].unique():
    try:
        os.makedirs(os.path.join('..', 'data', 'donor_tissue_celltype_split', celltype))    
        for filename in os.listdir('../data/donor_tissue_celltype_split'):
            if filename.endswith('.h5ad') and celltype in filename:
                os.rename(os.path.join('../data/donor_tissue_celltype_split', filename), os.path.join('../data/donor_tissue_celltype_split', celltype, filename))
    except FileExistsError as e:
        print(f'error caught: {e}')



In [3]:
sc.pp.filter_cells(adata_tf, min_genes=3) # filter cells with fewer than 3 genes
#sc.pp.filter_cells(adata_tf, min_counts=200)  # This is a weaker threshold than above. It is just to population the n_counts column in adata
sc.pp.filter_genes(adata_tf, min_cells=3) # filter genes detected in fewer than 3 cells

In [4]:
sc.write(os.path.join('..', 'data', 'adata_tf_cnmf.h5ad'), adata_tf)

## RUN CNMF

follow tutorial from here: https://github.com/dylkot/cNMF/blob/master/Tutorials/analyze_pbmc_example_data.ipynb

run it using the run_cnmf_dtc.py script and the run_cnmf_dtc.sh shell script.

## Analyze Results

### Fibroblasts

In [6]:
selected_k = 11
density_threshold = 2.00

In [9]:
dtc = 'tf_TSP1_Bladder_fibroblast'
with open(os.path.join('..', 'results', 'cnmf', 'tf_cNMF_dtc', 'fibroblast', dtc, 'cnmf_obj_after_combine.pkl'), 'rb') as f:
    cnmf_obj = pickle.load(f)

cnmf_obj.consensus(k=selected_k, density_threshold=density_threshold, show_clustering=True, close_clustergram_fig=False)

FileNotFoundError: [Errno 2] No such file or directory: '../../results/cnmf/tf_cNMF_dtc/fibroblast/tf_TSP1_Bladder_fibroblast/cnmf_tmp/tf_TSP1_Bladder_fibroblast.spectra.k_11.merged.df.npz'